# CNN-LSTM Model

In [131]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

In [132]:
np.random.seed(42)

Import the `initial_variables.csv` dataset

In [140]:
X = pd.read_csv('../csv/X.csv', index_col='date', parse_dates=True, infer_datetime_format=True)
y = pd.read_csv('../csv/y.csv', index_col='date', parse_dates=True, infer_datetime_format=True)

display(X.tail(), y.tail())

,open,high,low,close,volume,trend,rsi,rsi_fast_k,rsi_fast_d,williams_r,...,stk_visa,stk_wmt,stk_xom,usd_aud,usd_cad,usd_cny,usd_eur,usd_hkd,usd_jpy,day_of_week
date,,,,,,,,,,,,,,,,,,,,,
2022-07-25,3965.719971,3975.300049,3943.459961,3966.840088,2988650000,3865.392707,52.589569,26.460757,66.275194,-15.676524,...,214.270004,132.020004,89.980003,1.448290,1.29246,6.7505,0.98037,7.84944,136.311996,0
2022-07-26,3953.219971,3953.219971,3910.739990,3921.050049,3083420000,3871.253815,49.880434,0.000000,32.941861,-31.418430,...,212.490005,121.980003,89.629997,1.438600,1.28507,6.7495,0.97800,7.84913,136.431000,1
2022-07-27,3951.429932,4039.560059,3951.429932,4023.610107,3584170000,3880.207597,55.308438,100.000000,42.153586,-5.015708,...,210.470001,126.589996,91.570000,1.439390,1.28737,6.7623,0.98721,7.84926,136.970001,2
2022-07-28,4026.129883,4078.949951,3992.969971,4072.429932,3882850000,3893.329737,57.608834,100.000000,66.666667,-1.824344,...,211.350006,129.750000,92.639999,1.429613,1.28171,6.7574,0.97950,7.84910,136.110992,3
2022-07-29,4087.330078,4140.149902,4079.219971,4130.290039,3817740000,3908.661969,60.166728,100.000000,100.000000,-2.355495,...,212.110001,132.050003,96.930000,1.428400,1.28070,6.7458,0.98113,7.84950,134.397003,4


,close
date,
2022-07-25,3921.050049
2022-07-26,4023.610107
2022-07-27,4072.429932
2022-07-28,4130.290039
2022-07-29,4118.629883


Check for null values

In [139]:
print(X.isnull().values.any())
print(y.isnull().values.any())

False
False


### Split data into train and test sets

In [99]:
X = X.to_numpy()
y = y.to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

In [144]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=1, kernel_size=(1, 74), activation='tanh', input_shape=[1, 74, 1])
])